# 세션 1 – 채팅 부트스트랩 (Foundry Local)

이 노트북은 Foundry Local을 부트스트랩하고, 선호하는 모델 별칭을 다운로드하며, 표준 및 스트리밍 채팅 완료를 수행합니다.


# 시나리오
이 세션에서는 Foundry Local을 통해 로컬 소형 언어 모델이 응답하도록 설정하는 데 필요한 최소한의 절차를 소개합니다. 여러분은 다음을 수행하게 됩니다:
- SDK 및 클라이언트 종속성을 설치합니다.
- 선택한 별칭(기본값: `phi-3.5-mini`)에 대해 Foundry Local 매니저를 초기화합니다.
- 모델 메타데이터의 선택적 필드를 허용하기 위해 방어적인 몽키 패치를 적용합니다.
- 표준 채팅 완료 요청을 보냅니다.
- 응답을 토큰 단위로 스트리밍합니다.

목표는 RAG, 라우팅 또는 에이전트로 이동하기 전에 로컬 런타임 및 네트워크 경로를 검증하는 것입니다.


### 설명: 의존성 설치
이 최소 채팅 흐름에 필요한 Python 패키지를 설치합니다:
- `foundry-local-sdk`: 로컬 모델 및 서비스 라이프사이클 관리.
- `openai`: 채팅 완성을 위한 익숙한 클라이언트 추상화.
- `rich`: 노트북 출력물을 더 명확하게 보여주는 예쁜 출력.

다시 실행해도 안전합니다(멱등성). 이미 환경에 설치되어 있다면 건너뛰어도 됩니다.


In [1]:
# Install required libraries (idempotent)
!pip install -q foundry-local-sdk openai rich

### 설명: 핵심 임포트
노트북 전체에서 사용되는 모듈을 가져옵니다:
- `FoundryLocalManager`는 로컬 모델 런타임과 인터페이스를 제공합니다.
- `OpenAI` 클라이언트를 사용하여 익숙한 채팅 완료 API를 재사용할 수 있습니다.
- `rich.print`는 스타일이 적용된 출력을 제공합니다.

여기서는 네트워크 호출이 발생하지 않습니다—단지 네임스페이스를 준비하는 과정입니다.


In [2]:
import os
from foundry_local import FoundryLocalManager
from openai import OpenAI
from rich import print

### 설명: 매니저 초기화 및 메타데이터 패치
선택한 별칭에 대해 `FoundryLocalManager`를 초기화하고, `promptTemplate`이 `null`일 수 있는 서비스 응답을 유연하게 처리하기 위해 방어적인 몽키 패치를 적용합니다.

주요 결과:
- 서비스 상태와 엔드포인트를 확인합니다.
- 캐시된 모델을 나열합니다 (로컬 저장소 확인).
- 별칭에 대한 구체적인 모델 ID를 해결합니다 (이후 채팅 호출에 사용).

서비스 메타데이터에서 원시 검증 문제가 발생할 경우, SDK를 포크하지 않고 이를 정리하는 방법을 보여줍니다.


In [3]:
# Catalog-safe manager initialization (handles null promptTemplate values)
import os
from foundry_local import FoundryLocalManager
from foundry_local.models import FoundryModelInfo
from openai import OpenAI
from rich import print

# Monkeypatch to tolerate service responses where promptTemplate is null
_original_from_list_response = FoundryModelInfo.from_list_response

def _safe_from_list_response(response):  # type: ignore
    try:
        if isinstance(response, dict) and response.get("promptTemplate") is None:
            # Normalize to empty dict so pydantic validation passes
            response["promptTemplate"] = {}
    except Exception as e:  # pragma: no cover
        print(f"[yellow]Warning: safe wrapper encountered issue normalizing promptTemplate: {e}[/yellow]")
    return _original_from_list_response(response)

# Apply patch only once
if getattr(FoundryModelInfo.from_list_response, "__name__", "") != "_safe_from_list_response":
    FoundryModelInfo.from_list_response = staticmethod(_safe_from_list_response)  # type: ignore

ALIAS = os.getenv('FOUNDRY_LOCAL_ALIAS', 'phi-3.5-mini')
manager = FoundryLocalManager(ALIAS)
print(f'[bold green]Service running:[/bold green] {manager.is_service_running()}')
print(f'Endpoint: {manager.endpoint}')
print('Cached models:', manager.list_cached_models())
model_id = manager.get_model_info(ALIAS).id
print(f'Using model id: {model_id}')

Service running: True

Endpoint: http://127.0.0.1:57127/v1

Cached models:
[
    FoundryModelInfo(
        alias='gpt-oss-20b',
        id='gpt-oss-20b-cuda-gpu:1',
        version='1',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/gpt-oss-20b-cuda-gpu/versions/1',
        file_size_mb=9882,
        prompt_template={},
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-3.5-mini',
        id='Phi-3.5-mini-instruct-cuda-gpu:1',
        version='1',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-3.5-mini-instruct-cuda-gpu/versions/1',
        file_size_mb=2181,
        prompt_template={
            'prompt': '<|user|>\n{Content}<|end|>\n<|assistant|>',
            'assistant': '<|assistant|>\n{Content}<|end|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='phi-4-mini',
        id='Phi-4-mini-instruct-cuda-gpu:4',
        version='4',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/Phi-4-mini-instruct-cuda-gpu/versions/4',
        file_size_mb=3686,
        prompt_template={
            'system': '<|system|>{Content}<|end|>',
            'user': '<|user|>{Content}<|end|>',
            'assistant': '<|assistant|>{Content}<|end|>',
            'prompt': '<|user|>{Content}<|end|><|assistant|>'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='MIT',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-0.5b',
        id='qwen2.5-0.5b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-0.5b-instruct-cuda-gpu/versions/3',
        file_size_mb=528,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-7b',
        id='qwen2.5-7b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-7b-instruct-cuda-gpu/versions/3',
        file_size_mb=4843,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_start|>user\n{Content}<|im_end|>',
            'assistant': '<|im_start|>assistant\n{Content}<|im_end|>',
            'prompt': '<|im_start|>user\n{Content}<|im_end|>\n<|im_start|>assistant'
        },
        provider='AzureFoundry',
        publisher='Microsoft',
        license='apache-2.0',
        task='chat-completion',
        ep_override=None
    ),
    FoundryModelInfo(
        alias='qwen2.5-coder-7b',
        id='qwen2.5-coder-7b-instruct-cuda-gpu:3',
        version='3',
        execution_provider='CUDAExecutionProvider',
        device_type=<DeviceType.GPU: 'GPU'>,
        uri='azureml://registries/azureml/models/qwen2.5-coder-7b-instruct-cuda-gpu/versions/3',
        file_size_mb=4843,
        prompt_template={
            'system': '<|im_start|>system\n{Content}<|im_end|>',
            'user': '<|im_star

Using model id: Phi-4-mini-instruct-cuda-gpu:4

### 설명: 기본 채팅 완료
로컬 Foundry 엔드포인트를 가리키는 `OpenAI` 호환 클라이언트를 생성하고 단일 비스트리밍 채팅 완료를 수행합니다. 여기서 중점 사항:
- 모델이 오류 없이 응답하도록 보장합니다.
- 지연 시간 및 출력 형식을 검증합니다.
- 리소스를 절약하기 위해 `max_tokens`를 적당히 설정합니다.

이 작업이 실패할 경우, Foundry Local 서비스가 실행 중인지와 별칭이 올바르게 해석되는지 다시 확인하세요.


In [4]:
client = OpenAI(base_url=manager.endpoint, api_key=manager.api_key or 'not-needed')
prompt = 'List two benefits of local inference for privacy.'
resp = client.chat.completions.create(model=model_id, messages=[{'role':'user','content':prompt}], max_tokens=120, temperature=0.5)
print(resp.choices[0].message.content)

1. Enhanced Data Privacy: Local inference allows data to be processed locally on a user's device rather than being 
sent to a central server. This reduces the risk of sensitive information being intercepted or exposed during 
transmission, thereby enhancing privacy.

2. Reduced Data Collection: By performing inference locally, the need for extensive data collection is minimized. 
This means that less personal information needs to be stored or transmitted, further protecting user privacy.

### 설명: 스트리밍 채팅 완료
향상된 지연 시간 인식과 상호작용적인 사용자 경험을 위해 토큰 스트리밍을 시연합니다. 루프는 도착하는 대로 점진적인 델타를 출력합니다:
- 초기 부분 출력이 중요한 채팅 UI에 유용합니다.
- 전체 완료 지연 시간 대비 토큰 처리량을 측정할 수 있습니다.

이 패턴을 활용하여 토큰을 누적하거나, 진행 상태 위젯을 업데이트하거나, 생성 중간에 중단할 수 있습니다.


In [5]:
# Streaming example
stream = client.chat.completions.create(model=model_id, messages=[{'role':'user','content':'Give a one-sentence definition of edge AI.'}], stream=True, max_tokens=60, temperature=0.4)
for chunk in stream:
    delta = chunk.choices[0].delta
    if delta and delta.content:
        print(delta.content, end='', flush=True)
print()

Edge

AI

refers

to

artificial

...

with

a

central

server

.


---

**면책 조항**:  
이 문서는 AI 번역 서비스 [Co-op Translator](https://github.com/Azure/co-op-translator)를 사용하여 번역되었습니다. 정확성을 위해 최선을 다하고 있으나, 자동 번역에는 오류나 부정확성이 포함될 수 있습니다. 원본 문서의 원어 버전이 권위 있는 출처로 간주되어야 합니다. 중요한 정보의 경우, 전문적인 인간 번역을 권장합니다. 이 번역 사용으로 인해 발생하는 오해나 잘못된 해석에 대해 당사는 책임을 지지 않습니다.
